# websearch agent

In [6]:
from openai import OpenAI
import os
from dotenv import load_dotenv
from pocketflow import Node,Flow
import yaml
load_dotenv()

True

In [2]:
def call_llm(prompt):
    client = OpenAI(
        base_url= os.getenv("BASE_URL"),
        api_key=os.getenv("DASHSCOPE_API_KEY")
    )
    res = client.chat.completions.create(
        model = os.getenv("MODEL_NAME"),
        messages = [{
            "role": "user",
            "content": prompt
        }]
    )
    return res.choices[0].message.content

In [3]:
# 决定节点：作用是判断问题是直接返回回答还是继续search

class DecideAction(Node):
    def prep(self, shared):
        context = shared.get("context","没有之前的搜索")
        question = shared['question']

        return question, context
    
    def exec(self, prep_res):
        question, context = prep_res
        print("Agent 正在决定下一步步骤...")
        prompt = f"""
            ### 背景信息
            你是一名可以进行网络搜索的研究助手。
            问题：{question}
            以往研究：{context}
            ### 行动范围
            [1] search
            描述：在网络上查找更多信息
            参数：query（字符串）：要搜索的内容

            [2] answer
            描述：用现有知识回答问题
            参数：answer（字符串）：问题的最终答案
            
            ### 下一步行动
            根据背景信息和可用行动，决定下一步行动。
            请按以下格式返回你的回应：

            ```yaml
            thinking: |
                <你的分步推理过程>
            action: search 或 answer
            reason: <你选择该行动的原因>
            answer: <如果行动是answer，填写此处>
            search_query: <如果行动是search，填写具体的搜索查询>
            ```
            
            ###重要提示：请务必做到：
            所有多行字段使用适当的缩进（4 个空格）
            多行文本字段使用 | 字符
            单行字段不使用 | 字符
            """

        res = call_llm(prompt)
        yaml_str = res.split("```yaml")[1].split("```")[0].strip()

        return yaml.safe_load(yaml_str)

    def post(self, shared, prep_res, exec_res):
        if exec_res['action'] == 'search':
            shared['search_query'] = exec_res['search_query']
            print(f"Agent 决定去搜索：{shared['search_query']}")
        else:
            shared['context'] = exec_res['answer']
            print("Agent 决定回答答案")

        return exec_res['action']

In [4]:
# 进行网络搜索
def search_web(query: str):
    pass

In [10]:
# search节点：作用是搜索答案
class SearchWeb(Node):
    def prep(self, shared):
        return shared['search_query']
    

    def exec(self, prep_res):
        print(f"Agent 正在从网络搜索：{prep_res}")
        res = search_web(prep_res)

        return res
    

    def post(self, shared, prep_res, exec_res):
        previous = shared.get("context", "")
        shared["context"] = previous + "\n\nSEARCH:" + shared['search_query'] + "\nRESULT:" + exec_res
        print("Agent 搜索到数据，正在分析...")
        
        return 'decide'


In [ ]:
# 回答问题节点
class AnswerQuestion(Node):
    def prep(self, shared):
        return shared['question'], shared.get("context", "")
    
    def exec(self, prep_res):
        question, context = prep_res
        print("Agent 正在回答问题...")
        prompt = f"""
            ## 背景信息
            根据以下信息，回答问题。
            问题：{question}
            研究资料：{context}
            
            ## 你的回答：
            利用研究结果提供全面的答案。
            """
        
        answer = call_llm(prompt=prompt)
        return answer

    
    def post(self, shared, prep_res, exec_res):
        shared['answer'] = exec_res
        
        return 'done'

In [ ]:
def create_travel_agent_flow():
    decide_node = DecideAction()
    search_node = SearchWeb()
    answer_node = AnswerQuestion()

    # 表示当 decide_node 节点的决策结果为 "search" 时，工作流将跳转到 search_node 节点继续执行
    decide_node  - "search" >> search_node  

    # 表示当 decide_node 节点的决策结果为 "answer" 时，工作流将跳转到 answer_node 节点继续执行
    decide_node  - "answer" >> answer_node
    
    # 表示当 search_node 节点执行完毕后，返回 "decide" 信号，工作流将回到 decide_node 节点重新进行决策
    search_node  - "decide" >> decide_node

    return Flow(start=decide_node)

In [ ]:
def main():
    shared = {
        "travel_query": "广东工业大学有哪些校区"
    }
    travel_agent_flow = create_travel_agent_flow()
    travel_agent_flow.run(shared)
    print (shared.get ("answer", "俺不知道耶～😅"))

if __name__ == '__main__':
    main()